# P11.1

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
import numpy as np
from ipywidgets import interact, FloatSlider, Layout


class SISModelER:
    """
    SIS model class.
    """

    def __init__(self, num_nodes, probability, beta, delta):
        """
        Initialize the SIS model.

        Parameters:
        - num_nodes: Number of nodes in the graph.
        - probability: Probability of an edge between two nodes.
        - type: Type of graph (erdos_renyi or barabasi-albert).
        - m: Number of edges to attach, if barabasi-albert network chosen.

        Returns:
        - G: NetworkX graph.
        """
        self.G = nx.erdos_renyi_graph(num_nodes, probability)
        
        self.states_history = None
        self.beta = beta
        self.delta = delta
        self.degrees = np.array([self.G.degree[node] for node in self.G.nodes])
        self.avg_degree = np.sum(self.degrees)/len(self.G.nodes)
        self.epidemic_treshold = 1 / self.avg_degree

    def initialize_states(self, G, initial_infected_fraction):
        """
        Initialize the states of nodes in the graph.

        Parameters:
        - G: NetworkX graph.
        - initial_infected_fraction: Initial fraction of infected nodes.

        Returns:
        - states: Dictionary with node states (S for susceptible, I for infected).
        """
        states = {node: 'S' for node in G.nodes}
        node_list = list(G.nodes)  
        infected_nodes = random.sample(node_list, int(initial_infected_fraction * G.number_of_nodes()))
        for node in infected_nodes:
            states[node] = 'I'
        self.epidemic_treshold = 1 / ((self.avg_degree)*(len(states) - len(infected_nodes))/len(states))
        return states


    def simulate(self, initial_infected_fraction, num_steps):
        """
        Simulate the SIS model on an Erdős-Rényi graph.

        Parameters:
        - G: NetworkX graph.
        - initial_infected_fraction: Initial fraction of infected nodes.
        - beta: Infection probability.
        - delta: Recovery probability.
        - num_steps: Number of simulation steps.

        Returns:
        - states_history: List of dictionaries representing the states of nodes at each step.
        """
        self.states_history = []
        states = self.initialize_states(self.G, initial_infected_fraction)
        self.states_history.append(states.copy())

        for _ in range(num_steps):
            new_states = states.copy()

            for node in self.G.nodes:
                if states[node] == 'S':
                    infected_neighbors = [neighbor for neighbor in self.G.neighbors(node) if states[neighbor] == 'I']
                    infection_prob = 1 - (1 - self.beta) ** len(infected_neighbors)
                    
                    if random.random() < infection_prob:
                        new_states[node] = 'I'
                else:
                    if random.random() < self.delta:
                        new_states[node] = 'S'

            states = new_states
            self.states_history.append(states.copy())

        return self.states_history

    def plot_infection_curve(self):
        """
        Plot the infection curve of the SIS model simulation.
        """
        num_steps = len(self.states_history)
        num_infected = [len([node for node, state in self.states_history[step].items() if state == 'I']) for step in range(num_steps)]

        plt.figure(figsize=(12, 6))
        plt.plot(range(num_steps), np.array(num_infected)/len(self.G.nodes), label='Fraction of infected nodes')
        plt.xlabel('Simulation step')
        plt.ylabel('Number of infected nodes')
        if self.beta / self.delta >= self.epidemic_treshold:
            plt.title('Epidemic threshold exceeded', color='r')
            plt.axhline(1- ((self.delta/self.beta)* 1/self.avg_degree), color='r', linestyle='--', label='Theoretical limit value')
        else:
            plt.title('Epidemic threshold not exceeded', color='g')
            plt.axhline(0, color='r', linestyle='--', label='Theoretical limit value')
        plt.legend()
        plt.show()

    def plot_simulation(self, output_file='.\\simulations\\sis_simulation2.gif', fps=2):
        """
        Save the SIS model simulation results as a GIF.

        Parameters:
        - states_history: List of dictionaries representing the states of nodes at each step.
        - G: NetworkX graph.
        - output_file: Name of the output GIF file.
        """
        num_steps = len(self.states_history)

        fig, ax = plt.subplots(figsize=(12, 12))
        pos = nx.spring_layout(self.G)

        def update(step):
            ax.clear()
            infected_nodes = [node for node, state in self.states_history[step].items() if state == 'I']
            nx.draw(self.G, pos=pos, ax=ax, node_color=['r' if node in infected_nodes else 'b' for node in self.G.nodes])
            ax.set_title(f"Step {step}")

        ani = animation.FuncAnimation(fig, update, frames=num_steps, interval=500, repeat=False)
        ani.save(output_file, writer='pillow', fps=fps)




    def interact_simulation(self):
        """
        Interactively simulate the SIS model by manipulating beta and delta.
        """
        style = {'description_width': '400px'}

        interact(self._simulate_interactive,
                 beta=FloatSlider(min=0, max=0.3, step=0.001, value=self.beta, description='Infection prob:', style=style,
                                  layout=Layout(width='50%')), 
                 delta=FloatSlider(min=0, max=0.5, step=0.001, value=self.delta, description='Recovery prob:', style=style,
                                   layout=Layout(width='50%'))) 


    def _simulate_interactive(self, beta, delta):
        """
        Helper function for interactive simulation.
        """
        self.beta = beta
        self.delta = delta
        self.simulate(initial_infected_fraction=0.1, num_steps=300)
        self.plot_infection_curve()

In [2]:
sis_model_er = SISModelER(num_nodes=100, probability=0.1, beta=0.1, delta=0.1)
sis_model_er.interact_simulation()

interactive(children=(FloatSlider(value=0.1, description='Infection prob:', layout=Layout(width='50%'), max=0.…

In [61]:
print('Epidemic threshold', sis_model_er.epidemic_treshold) 

Epidemic threshold 0.10745755426606489


## Simulation

In [2]:
num_nodes = 100
probability = 0.1
initial_infected_fraction = 0.1
beta = 0.03 
delta = 0.01  
num_steps = 100

sis_model = SISModelER(num_nodes, probability, beta=beta, delta=delta)
sis_model.simulate(initial_infected_fraction, num_steps)
sis_model.plot_simulation(output_file='.\\simulations\\sis_simulation_erdos_renyi3.gif', fps=5)

# P11.2

In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
import numpy as np
from ipywidgets import interact, FloatSlider, Layout


class SISModelBA:
    """
    SIS model class for barabasi-albert.
    """

    def __init__(self, num_nodes, m, beta, delta):
        """
        Initialize the SIS model.

        Parameters:
        - num_nodes: Number of nodes in the graph.
        - probability: Probability of an edge between two nodes.
        - type: Type of graph (erdos_renyi or barabasi-albert).
        - m: Number of edges to attach, if barabasi-albert network chosen.

        Returns:
        - G: NetworkX graph.
        """
        
        self.G = nx.barabasi_albert_graph(num_nodes, m)
        
        
        self.states_history = None
        self.beta = beta
        self.delta = delta
        self.degrees = np.array([self.G.degree[node] for node in self.G.nodes])
        self.avg_degree = np.sum(self.degrees)/len(self.G.nodes)
        self.epidemic_treshold = np.sum(self.degrees) / np.sum(self.degrees**2)

    def initialize_states(self, G, initial_infected_fraction):
        """
        Initialize the states of nodes in the graph.

        Parameters:
        - G: NetworkX graph.
        - initial_infected_fraction: Initial fraction of infected nodes.

        Returns:
        - states: Dictionary with node states (S for susceptible, I for infected).
        """
        states = {node: 'S' for node in G.nodes}
        node_list = list(G.nodes) 
        infected_nodes = random.sample(node_list, int(initial_infected_fraction * G.number_of_nodes()))
        for node in infected_nodes:
            states[node] = 'I'
        return states


    def simulate(self, initial_infected_fraction, num_steps):
        """
        Simulate the SIS model on an Erdős-Rényi graph.

        Parameters:
        - G: NetworkX graph.
        - initial_infected_fraction: Initial fraction of infected nodes.
        - beta: Infection probability.
        - delta: Recovery probability.
        - num_steps: Number of simulation steps.

        Returns:
        - states_history: List of dictionaries representing the states of nodes at each step.
        """
        self.states_history = []
        states = self.initialize_states(self.G, initial_infected_fraction)
        self.states_history.append(states.copy())

        for _ in range(num_steps):
            new_states = states.copy()

            for node in self.G.nodes:
                if states[node] == 'S':
                    infected_neighbors = [neighbor for neighbor in self.G.neighbors(node) if states[neighbor] == 'I']
                    infection_prob = 1 - (1 - self.beta) ** len(infected_neighbors)
                    
                    if random.random() < infection_prob:
                        new_states[node] = 'I'
                else:
                    if random.random() < self.delta:
                        new_states[node] = 'S'

            states = new_states
            self.states_history.append(states.copy())

        return self.states_history

    def plot_infection_curve(self):
        """
        Plot the infection curve of the SIS model simulation.
        """
        num_steps = len(self.states_history)
        num_infected = [len([node for node, state in self.states_history[step].items() if state == 'I']) for step in range(num_steps)]

        plt.figure(figsize=(12, 6))
        plt.plot(range(num_steps), np.array(num_infected)/len(self.G.nodes), label='Fraction of infected nodes')
        plt.xlabel('Simulation step')
        plt.ylabel('Number of infected nodes')
        if self.beta / self.delta >= self.epidemic_treshold:
            # red title 
            plt.title('Epidemic threshold exceeded', color='r')
        else:
            # green title
            plt.title('Epidemic threshold not exceeded', color='g')
        plt.legend()
        plt.show()

    def plot_simulation(self, output_file='.\\simulations\\sis_simulation2.gif', fps=2):
        """
        Save the SIS model simulation results as a GIF.

        Parameters:
        - states_history: List of dictionaries representing the states of nodes at each step.
        - G: NetworkX graph.
        - output_file: Name of the output GIF file.
        """
        num_steps = len(self.states_history)

        fig, ax = plt.subplots(figsize=(12, 12))
        # initialize positions for nodes in the graph
        pos = nx.spring_layout(self.G)

        def update(step):
            ax.clear()
            infected_nodes = [node for node, state in self.states_history[step].items() if state == 'I']
            nx.draw(self.G, pos=pos, ax=ax, node_color=['r' if node in infected_nodes else 'b' for node in self.G.nodes])
            ax.set_title(f"Step {step}")

        ani = animation.FuncAnimation(fig, update, frames=num_steps, interval=500, repeat=False)
        ani.save(output_file, writer='pillow', fps=fps)




    def interact_simulation(self):
        """
        Interactively simulate the SIS model by manipulating beta and delta.
        """
        # bigger description width
        style = {'description_width': '400px'}

        interact(self._simulate_interactive,
                 beta=FloatSlider(min=0, max=0.3, step=0.005, value=self.beta, description='Infection prob:', style=style,
                                  layout=Layout(width='50%')), 
                 delta=FloatSlider(min=0, max=0.5, step=0.005, value=self.delta, description='Recovery prob:', style=style,
                                   layout=Layout(width='50%'))) 


    def _simulate_interactive(self, beta, delta):
        """
        Helper function for interactive simulation.
        """
        self.beta = beta
        self.delta = delta
        self.simulate(initial_infected_fraction=0.1, num_steps=300)
        self.plot_infection_curve()

In [4]:
sis_model_ba = SISModelBA(num_nodes=1000, m=5, beta=0.1, delta=0.1)
sis_model_ba.interact_simulation()

interactive(children=(FloatSlider(value=0.1, description='Infection prob:', layout=Layout(width='50%'), max=0.…

In [60]:
print('Epidemic threshold', sis_model_ba.epidemic_treshold) 

Epidemic threshold 0.04940318960894521


# P11.3

In [5]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
import numpy as np
from ipywidgets import interact, FloatSlider, Layout


class SIRModel:
    """
    SIR model class.
    """

    def __init__(self, num_nodes, probability, beta, gamma):
        """
        Initialize the SIR model.

        Parameters:
        - num_nodes: Number of nodes in the graph.
        - probability: Probability of an edge between two nodes.

        Returns:
        - G: NetworkX graph.
        """
        self.G = nx.erdos_renyi_graph(num_nodes, probability)
        
        self.states_history = None
        self.beta = beta
        self.gamma = gamma
        self.avg_degree = np.sum(np.array([self.G.degree[node] for node in self.G.nodes]))/len(self.G.nodes)
        self.epidemic_threshold = self.gamma / self.beta

    def initialize_states(self, G, initial_infected_fraction):
        """
        Initialize the states of nodes in the graph.

        Parameters:
        - G: NetworkX graph.
        - initial_infected_fraction: Initial fraction of infected nodes.

        Returns:
        - states: Dictionary with node states (S for susceptible, I for infected, R for recovered).
        """
        states = {node: 'S' for node in G.nodes}
        node_list = list(G.nodes) 
        infected_nodes = random.sample(node_list, int(initial_infected_fraction * G.number_of_nodes()))
        for node in infected_nodes:
            states[node] = 'I'
        return states

    def simulate(self, initial_infected_fraction, num_steps):
        """
        Simulate the SIR model on an Erdős-Rényi graph.

        Parameters:
        - initial_infected_fraction: Initial fraction of infected nodes.
        - num_steps: Number of simulation steps.

        Returns:
        - states_history: List of dictionaries representing the states of nodes at each step.
        """
        self.states_history = []
        states = self.initialize_states(self.G, initial_infected_fraction)
        self.states_history.append(states.copy())

     
        for _ in range(num_steps):
            new_states = states.copy()

            for node in self.G.nodes:
                if states[node] == 'S':
                    infected_neighbors = [neighbor for neighbor in self.G.neighbors(node) if states[neighbor] == 'I']
                    infection_prob = 1 - (1 - self.beta) ** len(infected_neighbors)
                    if random.random() < infection_prob:
                        new_states[node] = 'I'
                elif states[node] == 'I':
                    if random.random() < self.gamma:
                        new_states[node] = 'R'

            states = new_states
            self.states_history.append(states.copy())

        return self.states_history

    def plot_infection_curve(self):
        """
        Plot the infection curve of the SIR model simulation.
        """
        num_steps = len(self.states_history)
        num_infected = [len([node for node, state in self.states_history[step].items() if state == 'I']) for step in range(num_steps)]
        num_recovered = [len([node for node, state in self.states_history[step].items() if state == 'R']) for step in range(num_steps)]

        plt.figure(figsize=(12, 6))
        plt.plot(range(num_steps), np.array(num_infected)/len(self.G.nodes), label='Fraction of infected nodes', color='r')
        plt.plot(range(num_steps), np.array(num_recovered)/len(self.G.nodes), label='Fraction of recovered nodes', color='g')
        plt.xlabel('Simulation step')
        plt.ylabel('Fraction of nodes')
        plt.title('SIR model simulation')
        plt.legend()
        plt.show()

    def plot_simulation(self, output_file='.\\simulations\\sir_simulation.gif', fps=2):
        """
        Save the SIR model simulation results as a GIF.

        Parameters:
        - output_file: Name of the output GIF file.
        """
        num_steps = len(self.states_history)

        fig, ax = plt.subplots(figsize=(12, 12))
        pos = nx.spring_layout(self.G)

        def update(step):
            ax.clear()
            infected_nodes = [node for node, state in self.states_history[step].items() if state == 'I']
            recovered_nodes = [node for node, state in self.states_history[step].items() if state == 'R']
            nx.draw(self.G, pos=pos, ax=ax, node_color=['r' if node in infected_nodes else 'g' if node in recovered_nodes else 'b' for node in self.G.nodes])
            ax.set_title(f"Step {step}")

        ani = animation.FuncAnimation(fig, update, frames=num_steps, interval=500, repeat=False)
        ani.save(output_file, writer='pillow', fps=fps)

    def interact_simulation(self):
        """
        Interactively simulate the SIR model by manipulating beta and gamma.
        """
        # Bigger description width
        style = {'description_width': '400px'}

        interact(self._simulate_interactive,
                 beta=FloatSlider(min=0, max=0.3, step=0.005, value=self.beta, description='Infection prob:', style=style,
                                  layout=Layout(width='50%')), 
                 gamma=FloatSlider(min=0, max=0.3, step=0.005, value=self.gamma, description='Recovery prob:', style=style,
                                   layout=Layout(width='50%'))) 

    def _simulate_interactive(self, beta, gamma):
        """
        Helper function for interactive simulation.
        """
        self.beta = beta
        self.gamma = gamma
        self.simulate(initial_infected_fraction=0.05, num_steps=100)
        self.plot_infection_curve()


In [6]:
sir_model = SIRModel(num_nodes=1000, probability=0.05, beta=0.05, gamma=0.05)
sir_model.interact_simulation()

interactive(children=(FloatSlider(value=0.05, description='Infection prob:', layout=Layout(width='50%'), max=0…